In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# load pictures from drive

import os
from PIL import Image

class_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 
               'L', 'O', 'P', 'Q', 'R', 'U', 'V', 'W', 'X', 'Y']

class_names_labels = {j:i for i,j in enumerate(class_names)}   ## for loop to go through the name
print(class_names_labels)
IMAGE_SIZE = (50, 50)

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'K': 9, 'L': 10, 'O': 11, 'P': 12, 'Q': 13, 'R': 14, 'U': 15, 'V': 16, 'W': 17, 'X': 18, 'Y': 19}


In [6]:
## LOAD DATA - load images and labels from folder

def load_data():
    images=[]
    labels=[]
    path = "/content/drive/MyDrive/TML_project/TMLgesture"
    for folder in os.listdir(path):
        if folder == ".DS_Store":
            continue
            
        label = class_names_labels[folder]
        for file in os.listdir(os.path.join(path,folder)):
            if file.split('.')[-1] in {'jpg','png', 'jpeg'}:
                img_path = os.path.join(path,folder,file)
                img = Image.open(img_path)
                img = img.convert('RGB')
                img = img.resize(IMAGE_SIZE)
                
                images.append(img)
                labels.append(label)
    images = np.array([np.array(i) for i in images])
    labels = np.array([np.array(i) for i in labels])
    return images,labels

x,y = load_data()

print(x.shape)
print(y.shape)

(2400, 50, 50, 3)
(2400,)


In [7]:
## TRAIN-TEST SPLIT

from sklearn.model_selection import train_test_split
# highest random_state 200
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train,x_validate,y_train,y_validate = train_test_split(x_train,y_train, test_size = 0.25, random_state=42)
# x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size=0.25, random_state=0)  

print(x_train.shape)
print(x_validate.shape)
print(x_test.shape)

(1440, 50, 50, 3)
(480, 50, 50, 3)
(480, 50, 50, 3)


In [8]:
#### FLATTEN IMAGE DATA - X

x_train_flat = x_train.reshape(-1, x_train.shape[1] * x_train.shape[2] * x_train.shape[3]) 
x_validate_flat = x_validate.reshape(-1, x_validate.shape[1] * x_validate.shape[2] * x_validate.shape[3]) 
x_test_flat = x_test.reshape(-1, x_test.shape[1] * x_test.shape[2] * x_test.shape[3])

print(x_train_flat.shape, x_test_flat.shape, x_test_flat.shape)


(1440, 7500) (480, 7500) (480, 7500)


In [9]:
## feature scaling

x_train_scale = x_train_flat / 255
x_validate_scale = x_validate_flat / 255
x_test_scale = x_test_flat / 255

print(x_train_scale.shape)
print(x_validate_scale.shape)
print(x_test_scale.shape)

(1440, 7500)
(480, 7500)
(480, 7500)


In [10]:
# APPLY LDA

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Create an instance of LDA
lda = LinearDiscriminantAnalysis(n_components=19)

# Fit the LDA model with x_train data
lda.fit(x_train_scale, y_train)

# Transform the dataset using LDA
x_train_lda = lda.transform(x_train_scale)
x_validate_lda = lda.transform(x_validate_scale)
x_test_lda = lda.transform(x_test_scale)

print(x_train_lda.shape)
print(x_validate_lda.shape)
print(x_test_lda.shape)

(1440, 19)
(480, 19)
(480, 19)


In [11]:
## APPLY PCA

from sklearn.decomposition import PCA

# Create an instance of PCA
pca = PCA(n_components=20)

# Fit the PCA model with x_train data
pca.fit(x_train_scale)


# Transform the dataset using PCA
x_train_pca = pca.transform(x_train_scale)
x_validate_pca = pca.transform(x_validate_scale)
x_test_pca = pca.transform(x_test_scale)

print(x_train_pca.shape)
print(x_validate_pca.shape)
print(x_test_pca.shape)

(1440, 20)
(480, 20)
(480, 20)


GRID SEARCH

In [ ]:
# Grid search
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for grid search
param_grid = {
    'hidden_layer_sizes': [(50, 20), (100, 50), (200, 100), (300,200)],
    'alpha': [0.0001, 0.0005],
    'max_iter': [2000, 5000]
}

# Create an instance of the MLPClassifier
mlp_none = MLPClassifier()

# Create an instance of GridSearchCV
grid_search = GridSearchCV(mlp, param_grid, cv=5, scoring='accuracy')

# Perform grid search on the training data
grid_search.fit(x_train_scale, y_train)

# Print the best hyperparameters and the corresponding accuracy score
print("Best Hyperparameters: ", grid_search.best_params_)
print("Best Accuracy: ", grid_search.best_score_)

In [12]:
#### MLP Classification to train model -- PCA

from sklearn.neural_network import MLPClassifier
mlp_pca = MLPClassifier(hidden_layer_sizes=(200, 100), max_iter=5000, 
                 random_state=42, alpha=0.0001)

mlp_pca.fit(x_train_pca, y_train)
pca_pred = mlp_pca.predict(x_test_pca)
print("Training accuracy :", mlp_pca.score(x_train_pca, y_train))
print("Validating accuracy :", mlp_pca.score(x_validate_pca, y_validate))
print("Testing accuracy :", mlp_pca.score(x_test_pca, y_test))

Training accuracy : 1.0
Validating accuracy : 0.73125
Testing accuracy : 0.7395833333333334


In [13]:
#### MLP Classification to train model -- LDA

from sklearn.neural_network import MLPClassifier
mlp_lda = MLPClassifier(hidden_layer_sizes=(200, 100), max_iter=5000, 
                 random_state=42, alpha=0.0001)

mlp_lda.fit(x_train_lda, y_train)
lda_pred = mlp_lda.predict(x_test_lda)
print("Training accuracy :", mlp_lda.score(x_train_lda, y_train))
print("Validating accuracy :", mlp_lda.score(x_validate_lda, y_validate))
print("Testing accuracy :", mlp_lda.score(x_test_lda, y_test))

Training accuracy : 1.0
Validating accuracy : 0.5083333333333333
Testing accuracy : 0.5395833333333333


In [14]:
#### MLP Classification to train model -- None

from sklearn.neural_network import MLPClassifier

mlp_none = MLPClassifier(hidden_layer_sizes=(200, 100), max_iter=5000,
                         random_state=42, alpha=0.0001)

# Fit the model
mlp_none.fit(x_train_scale, y_train)

none_pred = mlp_none.predict(x_test_scale)

print("Training accuracy:", mlp_none.score(x_train_scale, y_train))
print("Validation accuracy:", mlp_none.score(x_validate_scale, y_validate))
print("Testing accuracy:", mlp_none.score(x_test_scale, y_test))


Training accuracy: 0.9166666666666666
Validation accuracy: 0.6083333333333333
Testing accuracy: 0.65


In [16]:
### Adaboost -- Bagging with MLP-PCA

from sklearn.ensemble import BaggingClassifier


# Create an instance of the BaggingClassifier with MLP as the base estimator
bagging = BaggingClassifier(base_estimator=mlp_pca, n_estimators=100, random_state=42)

# Fit the AdaBoost classifier to the training data
bagging.fit(x_train_pca, y_train)

# Predict on the test data
bagging_pred = bagging.predict(x_test_pca)

# Evaluate the accuracy of the model
accuracy = bagging.score(x_test_pca, y_test)
print("Bagging-PCA Accuracy:", accuracy)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Bagging-PCA Accuracy: 0.7583333333333333


In [18]:
### Adaboost -- Bagging with MLP-LDA

# Create an instance of the BaggingClassifier with MLP as the base estimator
bagging = BaggingClassifier(base_estimator=mlp_lda, n_estimators=100, random_state=42)

# Fit the AdaBoost classifier to the training data
bagging.fit(x_train_lda, y_train)

# Predict on the test data
bagging_pred = bagging.predict(x_test_lda)

# Evaluate the accuracy of the model
accuracy = bagging.score(x_test_lda, y_test)
print("Bagging-LDA Accuracy:", accuracy)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Bagging-LDA Accuracy: 0.5833333333333334


In [ ]:
### Adaboost -- Bagging with MLP-None

# Create an instance of the BaggingClassifier with MLP as the base estimator
bagging = BaggingClassifier(base_estimator=mlp_none, n_estimators=100, random_state=42)

# Fit the AdaBoost classifier to the training data
bagging.fit(x_train_scale, y_train)

# Predict on the test data
bagging_pred = bagging.predict(x_test_scale)

# Evaluate the accuracy of the model
accuracy = bagging.score(x_test_scale, y_test)
print("Bagging-None Accuracy:", accuracy)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


In [ ]:
## classification report for MLP-PCA

from sklearn.metrics import classification_report

print(classification_report(y_test, pca_pred, labels=mlp.classes_.tolist()))

In [ ]:
## confusion matrix -- for MLP-PCA

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_test, pca_pred, labels=mlp.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=mlp.classes_)

fig, ax = plt.subplots(figsize=(10,10))

ax.set_title("MLP-PCA Classifier")
disp.plot(ax=ax)
plt.show() 

In [ ]:
## classification report for MLP-LDA

from sklearn.metrics import classification_report

print(classification_report(y_test, lda_pred, labels=mlp.classes_.tolist()))

In [ ]:
## confusion matrix -- for MLP-LDA

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_test, lda_pred, labels=mlp.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=mlp.classes_)

fig, ax = plt.subplots(figsize=(10,10))

ax.set_title("MLP-LDA Classifier")
disp.plot(ax=ax)
plt.show() 

In [ ]:
## classification report for MLP-None

from sklearn.metrics import classification_report

print(classification_report(y_test, none_pred, labels=mlp.classes_.tolist()))

In [ ]:
## confusion matrix -- for MLP-None

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_test, none_pred, labels=mlp.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=mlp.classes_)

fig, ax = plt.subplots(figsize=(10,8))

ax.set_title("MLP-None Classifier")
sns.heatmap(cm, annot=True, cmap="YlGn")
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show() 

In [ ]:
## classification report for Bagging Meta


print(classification_report(y_test, bagging_pred, labels=bagging.classes_.tolist()))

In [ ]:
## confusion matrix -- for Bagging Classifier

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_test, bagging_pred, labels=bagging.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=bagging.classes_)

fig, ax = plt.subplots(figsize=(10,10))

ax.set_title("Bagging Classifier")
disp.plot(ax=ax)
plt.show() 